In [1]:
import dask_awkward as dak
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import dask_awkward as dak
import numpy as np
import json
import argparse
import os
from distributed import Client
import time    
import tqdm
import mplhep as hep
import matplotlib.pyplot as plt
import matplotlib
import glob
hep.style.use("CMS")
# client =  Client(n_workers=20,  threads_per_worker=2, processes=True, memory_limit='10 GiB') 
from dask_gateway import Gateway
gateway = Gateway(
    "http://dask-gateway-k8s.geddes.rcac.purdue.edu/",
    proxy_address="traefik-dask-gateway-k8s.cms.geddes.rcac.purdue.edu:8786",
)
cluster_info = gateway.list_clusters()[0]# get the first cluster by default. There only should be one anyways
client = gateway.connect(cluster_info.name).get_client()
print("Gateway Client created")

Gateway Client created


2024-05-09 15:11:43,693 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [2]:
# get data events
# common_load_path = "/depot/cms/users/yun79/results/stage1/Run2StorageTest/2018/f1_0"
common_load_path = "/work/users/yun79/stage1_output/Run2StorageTest/2018/f1_0"
data_load_path = common_load_path+"/data*/*/*.parquet"
# data_load_path = common_load_path+"/data_C/*/*.parquet"

data_events = dak.from_parquet(data_load_path) 
# we're only interested in ZCR
filter = ak.fill_none(data_events["z_peak"], value=False)
data_events = data_events[filter]



In [3]:
# get MC events (all MC except DY)
secondary_MC_strs = [
    "ttjet*", # all TT samples
    "st_tw*", # single Top samples
    "ewk*", # single Top samples
    "ww*", # diboson samples
    "wz*", # diboson samples
    "zz*", # diboson samples
]
filelist_big = []
for mc_str in secondary_MC_strs:
    filelist = glob.glob(common_load_path + f"/{mc_str}/*/*.parquet")
    filelist_big += filelist
mc_events = dak.from_parquet(filelist_big) 
# we're only interested in ZCR
filter = ak.fill_none(mc_events["z_peak"], value=False)
mc_events = mc_events[filter]

In [4]:
def get_calib_categories(events):
    BB = ((abs(events["mu1_eta"])<=0.9) & (abs(events["mu2_eta"])<=0.9))
    BO = ((abs(events["mu1_eta"])<=0.9) & ((abs(events["mu2_eta"])>0.9) & (abs(events["mu2_eta"]) <=1.8)))
    BE = ((abs(events["mu1_eta"])<=0.9) & ((abs(events["mu2_eta"])>1.8) & (abs(events["mu2_eta"]) <=2.4)))
    OB = (((abs(events["mu1_eta"])>0.9) & (abs(events["mu1_eta"]) <=1.8)) & (abs(events["mu2_eta"])<=0.9))
    OO = (((abs(events["mu1_eta"])>0.9) & (abs(events["mu1_eta"]) <=1.8)) & ((abs(events["mu2_eta"])>0.9) & (abs(events["mu2_eta"]) <=1.8)))
    OE = (((abs(events["mu1_eta"])>0.9) & (abs(events["mu1_eta"]) <=1.8)) & ((abs(events["mu2_eta"])>1.8) & (abs(events["mu2_eta"]) <=2.4)))
    EB = (((abs(events["mu1_eta"])>1.8) & (abs(events["mu1_eta"]) <=2.4)) & (abs(events["mu2_eta"])<=0.9))
    EO = (((abs(events["mu1_eta"])>1.8) & (abs(events["mu1_eta"]) <=2.4)) & ((abs(events["mu2_eta"])>0.9) & (abs(events["mu2_eta"]) <=1.8)))
    EE = (((abs(events["mu1_eta"])>1.8) & (abs(events["mu1_eta"]) <=2.4)) & ((abs(events["mu2_eta"])>1.8) & (abs(events["mu2_eta"]) <=2.4)))
    categories = [((events["mu1_pt"]>30)&(events["mu1_pt"]<=45)&(BB | OB | EB)),
                          ((events["mu1_pt"]>30)&(events["mu1_pt"]<=45)&(BO | OO | EO)),
                          ((events["mu1_pt"]>30)&(events["mu1_pt"]<=45)&(BE | OE | EE)),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&BB),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&BO),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&BE),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&OB),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&OO),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&OE),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&EB),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&EO),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&EE),
                  # voigtian start here onwards
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&BB),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&BO),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&BE),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&OB),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&OO),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&OE),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&EB),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&EO),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&EE),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&BB),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&BO),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&BE),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&OB),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&OO),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&OE),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&EB),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&EO),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&EE),]
    return categories

In [5]:
import time
time_step = time.time()
binning = np.linspace(*[70, 110, 100])
# fraction_weight = ak.to_numpy(1/events.fraction.compute())
data_categories = get_calib_categories(data_events)
mc_categories = get_calib_categories(mc_events)
for idx in range(len(data_categories)):
    cat_selection = data_categories[idx]
    # print(f"cat_selection: {cat_selection.compute()}")
    fraction_weight = 1/data_events.fraction
    weights = data_events.weights*fraction_weight*cat_selection
    # weights = ak.to_numpy(weights.compute())
    dimuon_mass = ak.fill_none(data_events.dimuon_mass, value=-999.0)
    # dimuon_mass = ak.to_numpy(dimuon_mass.compute())
    filter = (dimuon_mass!=-999.0) & (weights != 0)
    dimuon_mass = dimuon_mass[filter]
    weights = weights[filter]
    # data_hist, _ = np.histogram(dimuon_mass.compute(), bins=binning, weights = weights.compute())
    zip = ak.zip({
        "dimuon_mass" : dimuon_mass,
        "weights" : weights,
    })
    zip = zip.compute()
    data_hist, _ = np.histogram(zip.dimuon_mass, bins=binning, weights = zip.weights)
    # MC ------------------------------------------------------------------------------------
    cat_selection = mc_categories[idx]
    # print(f"cat_selection: {cat_selection.compute()}")
    fraction_weight = 1/mc_events.fraction
    weights = mc_events.weights*fraction_weight*cat_selection
    # weights = ak.to_numpy(weights.compute())
    dimuon_mass = ak.fill_none(mc_events.dimuon_mass, value=-999.0)
    # dimuon_mass = ak.to_numpy(dimuon_mass.compute())
    filter = (dimuon_mass!=-999.0) & (weights != 0)
    dimuon_mass = dimuon_mass[filter]
    weights = weights[filter]
    # mc_hist, _ = np.histogram(dimuon_mass.compute(), bins=binning, weights = weights.compute())
    zip = ak.zip({
        "dimuon_mass" : dimuon_mass,
        "weights" : weights,
    })
    zip = zip.compute()
    mc_hist, _ = np.histogram(zip.dimuon_mass, bins=binning, weights = zip.weights)

    fig, ax_main = plt.subplots()
    hep.histplot(
        data_hist, xerr=True, 
        # yerr=data_err,
        bins=binning, stack=False, histtype='errorbar', color='black', 
        label='Data', ax=ax_main
    )
    hep.histplot(mc_hist, bins=binning, 
                             stack=False, histtype='fill', 
                             label="MC", 
                             # sort='label_r', 
                             color="Green", 
                             ax=ax_main)
    ax_main.legend(loc="upper right")
    ax_main.set_xlabel("dimuon mass [GeV]")
    ax_main.set_title(f"Cat {idx} ZCR mass distribution")
    plt.savefig(f"./cat{idx}.pdf")
    ax_main.set_yscale('log')
    ax_main.set_ylim(0.01, 1e9)
    plt.savefig(f"./cat{idx}_log.pdf")
    print(f"time elapsed in : {time.time()-time_step}")
    time_step = time.time()
    plt.clf()


time elapsed in : 76.4024121761322
time elapsed in : 72.85243368148804
time elapsed in : 71.72584366798401


KeyboardInterrupt: 

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

In [7]:
for idx in range(30):
    name = f"cat{idx}.png"
    print("\includegraphics[width=0.35" +"\textwidth]{images/" +name+"}")

\includegraphics[width=0.35	extwidth]{images/cat0.png}
\includegraphics[width=0.35	extwidth]{images/cat1.png}
\includegraphics[width=0.35	extwidth]{images/cat2.png}
\includegraphics[width=0.35	extwidth]{images/cat3.png}
\includegraphics[width=0.35	extwidth]{images/cat4.png}
\includegraphics[width=0.35	extwidth]{images/cat5.png}
\includegraphics[width=0.35	extwidth]{images/cat6.png}
\includegraphics[width=0.35	extwidth]{images/cat7.png}
\includegraphics[width=0.35	extwidth]{images/cat8.png}
\includegraphics[width=0.35	extwidth]{images/cat9.png}
\includegraphics[width=0.35	extwidth]{images/cat10.png}
\includegraphics[width=0.35	extwidth]{images/cat11.png}
\includegraphics[width=0.35	extwidth]{images/cat12.png}
\includegraphics[width=0.35	extwidth]{images/cat13.png}
\includegraphics[width=0.35	extwidth]{images/cat14.png}
\includegraphics[width=0.35	extwidth]{images/cat15.png}
\includegraphics[width=0.35	extwidth]{images/cat16.png}
\includegraphics[width=0.35	extwidth]{images/cat17.png}
\i